In [1]:
import tensorflow as tf
import numpy as np
from osgeo import gdal
import os
import random
import pickle
import subprocess
import glob

In [2]:
WORKDIR = '/scripts/data/generated'
SESSIONDIR = '/scripts/data/sessions'
boarddir_train = '/scripts/tb_logs/train/8'
boarddir_test = '/scripts/tb_logs/test/8'
os.chdir(WORKDIR)

FileNotFoundError: [Errno 2] No such file or directory: '/scripts/data/generated'

In [3]:
subprocess.Popen(['tensorboard', '--logdir=../../tb_logs', '--port=6006'])
learning_rate = 0.0001
num_steps = 500
batch_size = 8
#display_step = 10
num_epochs = 10000

# Network Parameters
edgelength = 32
#num_pixels = edgelength * edgelength
num_chans = 4
#num_input = num_pixels * num_chans # MNIST data input (img shape: 28*28)
num_classes = 5 # MNIST total classes (0-9 digits)
dropout = .85 # Dropout, probability to keep units
fac=2

# tf Graph input
with tf.name_scope('Input'):
    X = tf.placeholder(tf.float32, [None, edgelength, edgelength, num_chans])
    tf.summary.image('input_images', X[:, :, :, 0:3][:, :, :, ::-1], max_outputs=4)
    Y = tf.placeholder(tf.float32, [None, num_classes])
    keep_prob = tf.placeholder(tf.float32) # dropout (keep probability)

In [4]:
# Create some wrappers for simplicity
def conv2d(x, W, b, strides=1):
    # Conv2D wrapper, with bias and relu activation
    x = tf.nn.conv2d(x, W, strides=[1, strides, strides, 1], padding='SAME')
    x = tf.nn.bias_add(x, b)
    return tf.nn.relu(x)


def maxpool2d(x, k=2):
    # MaxPool2D wrapper
    return tf.nn.max_pool(x, ksize=[1, k, k, 1], strides=[1, k, k, 1],
                          padding='SAME')

# Create model
def conv_net(x, weights, biases, dropout):
    # MNIST data input is a 1-D vector of 784 features (28*28 pixels)
    # Reshape to match picture format [Height x Width x Channel]
    # Tensor input become 4-D: [Batch Size, Height, Width, Channel]
    # x = tf.reshape(x, shape=[-1, 28, 28, 1])

    # Convolution Layer
    conv1 = conv2d(x, weights['wc1'], biases['bc1'])
    # Max Pooling (down-sampling)
    conv1 = maxpool2d(conv1, k=2)

    # Convolution Layer
    conv2 = conv2d(conv1, weights['wc2'], biases['bc2'])
    # Max Pooling (down-sampling)
    conv2 = maxpool2d(conv2, k=2)
    
    # Convolution Layer
    conv3 = conv2d(conv2, weights['wc3'], biases['bc3'])
    # Max Pooling (down-sampling)
    conv3 = maxpool2d(conv3, k=2)
    
    # Convolution Layer
    conv4 = conv2d(conv3, weights['wc4'], biases['bc4'])
    # Max Pooling (down-sampling)
    conv4 = maxpool2d(conv4, k=2)
    
    # Convolution Layer
    #conv5 = conv2d(conv4, weights['wc5'], biases['bc5'])
    # Max Pooling (down-sampling)
    #conv5 = maxpool2d(conv5, k=2)

    # Fully connected layer
    # Reshape conv2 output to fit fully connected layer input
    fc1 = tf.reshape(conv4, [-1, weights['wd1'].get_shape().as_list()[0]])
    fc1 = tf.add(tf.matmul(fc1, weights['wd1']), biases['bd1'])
    fc1 = tf.nn.relu(fc1)
    # Apply Dropout
    fc1 = tf.nn.dropout(fc1, dropout)

    # Output, class prediction
    out = tf.add(tf.matmul(fc1, weights['out']), biases['out'])
    return out

In [5]:
# Store layers weight & bias
weights = {
    # 5x5 conv, 1 input, 32 outputs
    'wc1': tf.Variable(tf.random_normal([3, 3, num_chans, int(32*fac)])),
    'wc2': tf.Variable(tf.random_normal([3, 3, int(32*fac), int(64*fac)])),
    'wc3': tf.Variable(tf.random_normal([3, 3, int(64*fac), int(128*fac)])),
    'wc4': tf.Variable(tf.random_normal([3, 3, int(128*fac), int(256*fac)])),
    #'wc5': tf.Variable(tf.random_normal([3, 3, 256, 512])),
    
    'wd1': tf.Variable(tf.random_normal([2*2*int(256*fac), int(512*fac)])),
    'out': tf.Variable(tf.random_normal([int(512*fac), num_classes]))
}

biases = {
    'bc1': tf.Variable(tf.random_normal([int(32*fac)])),
    'bc2': tf.Variable(tf.random_normal([int(64*fac)])),
    'bc3': tf.Variable(tf.random_normal([int(128*fac)])),
    'bc4': tf.Variable(tf.random_normal([int(256*fac)])),
    #'bc5': tf.Variable(tf.random_normal([512])),
    'bd1': tf.Variable(tf.random_normal([int(512*fac)])),
    'out': tf.Variable(tf.random_normal([num_classes]))
}

# Construct model
logits = conv_net(X, weights, biases, keep_prob)
prediction = tf.nn.softmax(logits)

# Define loss and optimizer

#Notice - we introduce our L2 here
vars_to_regul   = tf.trainable_variables() 
lossL2 = tf.add_n([ tf.nn.l2_loss(v) for v in vars_to_regul
                    if 'bias' not in v.name ]) * 0.001

#  loss = (tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
#    out_layer, tf_train_labels) +
#    beta*tf.nn.l2_loss(hidden_weights) +
#    beta*tf.nn.l2_loss(hidden_biases) +
#    beta*tf.nn.l2_loss(out_weights) +
#    beta*tf.nn.l2_loss(out_biases)))

loss_op = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(
    logits=logits, labels=Y)) + lossL2


optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
train_op = optimizer.minimize(loss_op)

# Evaluate model
correct_pred = tf.equal(tf.argmax(prediction, 1), tf.argmax(Y, 1))
accuracy = tf.reduce_mean(tf.cast(correct_pred, tf.float32))

tf.summary.scalar('accuracy', accuracy)
tf.summary.scalar('loss', loss_op)
merged = tf.summary.merge_all()

train_writer = tf.summary.FileWriter(boarddir_train)
test_writer = tf.summary.FileWriter(boarddir_test)

In [6]:
init = tf.global_variables_initializer()

In [7]:
def train_test_split(loc, p):
    y_files = glob.glob(loc + '/*y.tif')
    x_files = [f[:-6] + '.tif' for f in y_files]
    
    c = list(zip(y_files, x_files))
    random.shuffle(c)
    y_file, x_file = zip(*c)
    
    number = int(len(y_file) * p)
    
    x_file_tr = x_file[:number]
    y_file_tr = y_file[:number]
    
    x_file_te = x_file[number:]
    y_file_te = y_file[number:]
    
    return list(x_file_tr), list(y_file_tr), list(x_file_te), list(y_file_te)

def load_data_generate_batches(x_files_tr, y_files_tr, bs=8, edgelength=32, augment=True, centr=True):
 
    Xss = []
    yss = []
    
    for i, x in enumerate(x_files_tr):
        
        # load xs
        S = gdal.Open(x)
        A = S.ReadAsArray(0, 0, edgelength, edgelength) # define edgelength       
        where_are_NaNs = np.isnan(A)
        A[where_are_NaNs] = -40.            
        X = np.transpose(A)
        
        S_y = gdal.Open(y_files_tr[i])        
        if centr == True:
            y = S_y.ReadAsArray(int(edgelength/2), int(edgelength/2), 1, 1)[0][0] # central pixel            
        else:
            y = S_y.ReadAsArray(0, 0, edgelength, edgelength) # define edgelength also; also think about NoData
            
            
        Xss.append(X)
        yss.append(y)
        
        if augment == True:
            for r in range(0,3):
                Xss.append(np.rot90(X, r))
                Xss.append(np.fliplr(np.rot90(X, r)))
                yss.append(y)
                yss.append(y)
                
    # shuffle both synchronously
    
    c = list(zip(yss, Xss))
    random.shuffle(c)
    yss, Xss = zip(*c)
    
    # then...
    X_batch = []
    y_batch = []
    X_batches = []
    y_batches = []
    
    yss = list(yss)
    Xss = list(Xss)
    
    for i, x in enumerate(Xss):
        X_batch.append(x)
        y_batch.append(yss[i])
        
        if (i % bs == 0) and (i != 0):
            if centr == True:
                y_corr = np.array(y_batch)
                y_corr[y_corr == 255] = 0
                n_values = num_classes
                y_batch = np.eye(n_values)[y_corr]
            
            X_batches.append(X_batch)
            y_batches.append(y_batch)
            
            X_batch = []
            y_batch = []
            
    X_batches[0] = X_batches[0][:-1]
    y_batches[0] = y_batches[0][:-1]
    
    return X_batches, y_batches   
    

def generate_test(X_locs, y_locs, edgelength, centr=True):
    #print(enumerate(X_locs))
    Xss = []
    yss = []
    for i, x in enumerate(X_locs):
        S = gdal.Open(x)
        S_y = gdal.Open(y_locs[i])
        A = S.ReadAsArray(0, 0, edgelength, edgelength)
        where_are_NaNs = np.isnan(A)
        A[where_are_NaNs] = -40.
        A = np.transpose(A)
        
        if centr == True:
            y = S_y.ReadAsArray(int(edgelength/2), int(edgelength/2), 1, 1)[0][0]
        else:
            y = S_y.ReadAsArray(0, 0, edgelength, edgelength)
        
        Xss.append(A)
        yss.append(y)
    
    if centr == True:
        y_file = np.array(yss)
        #if np.min(y_file) < 0:
        #    print('hello')
        y_file[y_file == 255] = 0
        n_values = num_classes
        yss = np.eye(n_values)[y_file]
    return Xss, yss

def standardization_parameters(batches_of_x, test_x):
    X = np.array(batches_of_x)
    X_te = np.array(test_x)
    
    means = []
    sds = []
    for i in range(X.shape[4]):
        mean = np.mean(X[:,:,:,:,i])
        sd = np.std(X[:,:,:,:,i])
        X[:,:,:,:,i] = (X[:,:,:,:,i] - mean) / sd
        X_te[:,:,:,i] = (X_te[:,:,:,i] - mean) / sd
        means.append(mean)
        sds.append(sd)
        
    return list(X), list(X_te), means, sds

def batchload(WORKDIR, train_test_ratio, batch_size, edgelength, centr, ):
    xtr, ytr, xte, yte = train_test_split(WORKDIR, train_test_ratio)
    X_batches, y_batches = load_data_generate_batches(xtr,ytr,8,32,True,True)
    xte, yte = generate_test(xte,yte,32,True)
    X_batches, xte, MNs, SDs = standardization_parameters(X_batches, xte)
    return X_batches, y_batches, xte, yte, MNs, SDs, 

In [8]:
#os.chdir('./sumatra_tiles')
import glob
train_test_ratio = 0.85
xtr, ytr, xte, yte = train_test_split(WORKDIR, train_test_ratio)

In [9]:
X_batches, y_batches = load_data_generate_batches(xtr,ytr,8,32,True,True)

In [10]:
xte, yte = generate_test(xte,yte,32,True)

In [11]:
X_batches, xte, MNs, SDs = standardization_parameters(X_batches, xte)

In [12]:
# Start training
# step = 0
display_step = 1
saver = tf.train.Saver()
with tf.Session() as sess:
    sess.run(init)
    
    for e in range(1, num_epochs + 1):
        for i, xb in enumerate(X_batches):
            yb = list(y_batches[i])
            sess.run(train_op, feed_dict={X: xb, Y: yb, keep_prob: dropout}) 
            #print(i)
            
        if e % display_step == 0 or e == 1:
            summary, loss, acc = sess.run([merged, loss_op, accuracy], feed_dict={X: xb,
                                                                 Y: yb,
                                                                 keep_prob: 1.0})
            train_writer.add_summary(summary, e)

            print("Step " + str(e) + ", Minibatch Loss= " + \
                    "{:.4f}".format(loss) + ", Training Accuracy= " + \
                    "{:.3f}".format(acc))

            # Calculate accuracy for 256 MNIST test images
            print("Testing Accuracy:")
            summary, acc = sess.run([merged, accuracy], feed_dict={X: xte,
                                      Y: yte,
                                      keep_prob: 1.0})
            test_writer.add_summary(summary, e)
            print(acc)
            
            save_path = saver.save(sess, SESSIONDIR+"/model.ckpt")
            print("Model saved in file: %s" % save_path)

Step 1, Minibatch Loss= 261565.6719, Training Accuracy= 0.875
Testing Accuracy:
0.905579
Model saved in file: /scripts/data/sessions/model.ckpt
Step 2, Minibatch Loss= 188918.8281, Training Accuracy= 0.875
Testing Accuracy:
0.881974
Model saved in file: /scripts/data/sessions/model.ckpt
Step 3, Minibatch Loss= 315180.4062, Training Accuracy= 0.875
Testing Accuracy:
0.877682
Model saved in file: /scripts/data/sessions/model.ckpt
Step 4, Minibatch Loss= 224312.9062, Training Accuracy= 0.875
Testing Accuracy:
0.875537
Model saved in file: /scripts/data/sessions/model.ckpt
Step 5, Minibatch Loss= 1670.3136, Training Accuracy= 1.000
Testing Accuracy:
0.88412
Model saved in file: /scripts/data/sessions/model.ckpt
Step 6, Minibatch Loss= 132436.5938, Training Accuracy= 0.875
Testing Accuracy:
0.886266
Model saved in file: /scripts/data/sessions/model.ckpt
Step 7, Minibatch Loss= 1663.2891, Training Accuracy= 1.000
Testing Accuracy:
0.873391
Model saved in file: /scripts/data/sessions/model.ck

KeyboardInterrupt: 

In [16]:
S = gdal.Open('/data/Hdd1/CNNTEST.tif')
Xr = S.ReadAsArray()
where_are_NaNs = np.isnan(Xr)
Xr[where_are_NaNs] = -40.

Z = [(Xr[i,:,:] - MNs[i])/SDs[i] for i in range(len(MNs))]
Z = np.array(Z)
Z = np.transpose(Z)
#M = np.zeros(Xr[:,:,0].shape)

M = np.zeros(Z[:,:,0].shape)

In [17]:
# Add ops to save and restore all the variables.
saver = tf.train.Saver()

# Later, launch the model, use the saver to restore variables from disk, and
# do some work with the model.

with tf.Session() as sess:
    saver.restore(sess, SESSIONDIR+"/model.ckpt")
    
    for i in range(18,Z.shape[0]-18):
        for j in range(18,Z.shape[1]-18):
            #print(y)
            classify_this = Z[i-int(edgelength/2):i+int(edgelength/2),j-int(edgelength/2):j+int(edgelength/2),:]
            ct = np.expand_dims(classify_this, axis=0)           
            y_hat = sess.run(prediction, feed_dict={X: ct, Y: np.expand_dims(yb[0], axis=0), keep_prob:1.0})
            CLASS = np.argmax(y_hat)
            M[i,j] = CLASS
            #print(CLASS)
        print(i)
    
  # Restore variables from disk.
  

    
    
  #print("Model restored.")
  # Check the values of the variables
  #print("v1 : %s" % v1.eval())
  #print("v2 : %s" % v2.eval())

INFO:tensorflow:Restoring parameters from /scripts/data/sessions/model.ckpt
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
26

In [ ]:
M.transpose().shape

In [19]:
M.shape

(680, 590)

In [20]:
# Write output
S2 = gdal.Open('/data/Hdd1/CNNTEST.tif')
Xr2 = S2.ReadAsArray()
driver = gdal.GetDriverByName('Gtiff')
dstfile = '/scripts/data/lcmaptestttt99.tif'
dataset2 = driver.Create(dstfile, S2.RasterXSize, S2.RasterYSize, 1, gdal.GDT_Float32)
dataset2.SetGeoTransform(S2.GetGeoTransform())
dataset2.SetProjection(S2.GetProjection())
Mt = M.transpose()
dataset2.GetRasterBand(1).WriteArray(Mt)
dataset2.FlushCache()
dataset2 = None

In [ ]:
Mt.shape

In [ ]:
np.transpose(Xr).shape

In [ ]:
Xr.shape